<a href="https://colab.research.google.com/github/D3auuvc/ut-assignments/blob/main/Homeword4_CNN_Release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 4
### Text Classification with CNN

Welcome to Homework 4! 

The homework contains several tasks. You can find the amount of points that you get for the correct solution in the task header. Maximum amount of points for each homework is _four_.

The **grading** for each task is the following:
- correct answer - **full points**
- insufficient solution or solution resulting in the incorrect output - **half points**
- no answer or completely wrong solution - **no points**

Even if you don't know how to solve the task, we encourage you to write down your thoughts and progress and try to address the issues that stop you from completing the task.

When working on the written tasks, try to make your answers short and accurate. Most of the times, it is possible to answer the question in 1-3 sentences.

When writing code, make it readable. Choose appropriate names for your variables (`a = 'cat'` - not good, `word = 'cat'` - good). Avoid constructing lines of code longer than 100 characters (79 characters is ideal). If needed, provide the commentaries for your code, however, a good code should be easily readable without them :)

Finally, all your answers should be written only by yourself. If you copy them from other sources it will be considered as an academic fraud. You can discuss the tasks with your classmates but each solution must be individual.

<font color='red'>**Important!:**</font> **before sending your solution, do the `Kernel -> Restart & Run All` to ensure that all your code works.**

In [ ]:
!pip install datasets torchmetrics --quiet

In [ ]:
import torch
import torch.nn as nn
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import f1_score, accuracy
import json
from tqdm.notebook import tqdm

In this homework, you are going to work again with [Stanford Semantic Treebank](https://nlp.stanford.edu/sentiment/index.html).

Only this time, we are going to split the labels into five classes, instead of two. This way, we will do a multi-class classification.

Run the cell below to load the dataset.

In [ ]:
sst = load_dataset("sst", "default")

Download the pretrained [GloVe 6B](https://nlp.stanford.edu/projects/glove/) word embeddings. 

If you are on Windows, just copy the URL into your browser, download the ZIP file and unpack it into the same folder as this notebook.

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
# Load the embeddings into the memory
glove_path = 'glove.6B.300d.txt'
glove_vecs = []
idx2token = []
with open(glove_path, encoding='utf-8') as f:
    for line in tqdm(f):
        line = line.strip().split()
        word = line[0]
        vec = [float(x) for x in line[1:]]
        glove_vecs.append(vec)
        idx2token.append(word)

# Convert to tensor
glove_vecs = torch.tensor(glove_vecs)

# Put zero vector for padding and mean for unknown
glove_vecs = torch.vstack(
    [
        torch.zeros(1, glove_vecs.size(1)),
        torch.mean(glove_vecs, dim=0).unsqueeze(0),
        glove_vecs,
    ]
)

# Save the embeddings in Pytorch format
torch.save(glove_vecs, 'glove.6B.300d.pt')

# Add special pad and unk tokens to the vocab
PAD = '<pad>'
PAD_ID = 0
UNK = '<unk>'
UNK_ID = 1

idx2token = [PAD, UNK] + idx2token

# Save the vocab
json.dump(idx2token, open('idx2token.json', 'w', encoding='utf-8'))

Uncomment the cell below if you want to load the saved embeddings and vocabulary

In [ ]:
# PAD = '<pad>'
# PAD_ID = 0
# UNK = '<unk>'
# UNK_ID = 1

# glove_vecs = torch.load('glove.6B.300d.pt')
# idx2token = json.load(open('idx2token.json', encoding='utf-8'))

In [ ]:
# Create a reverse vocab
token2idx = {token: idx for idx, token in enumerate(idx2token)}

In [ ]:
# Set the device (GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### Task 1. Build the dataset (1 point)

The SST dataset has a sentiment label for each sentence. This label ranges from 0 to 1, 0 being very negative and 1 being very positive. During the practice session, we split the labels into a binary format, i.e. all the labels lower than 0.5 became 0 and greater than 0.5 became 1.

This time, you will need to split them into five categories that have the following ranges: $(0, 0.2]$, $(0.2, 0.4]$, $(0.4, 0.6]$, $(0.6, 0.8]$, $(0.8, 1]$. They should be transformed into the labels $0$, $1$, $2$, $3$, $4$ respectively.

In the end, you should have five labels:
- 0 (very negative)
- 1 (negative)
- 2 (neural)
- 3 (positive)
- 4 (very positive)

In [ ]:
class SSTDataset(Dataset):
    def __init__(self, data, token2idx, max_seq_len=100, device=torch.device('cpu')):
        super().__init__()
        self.max_seq_len = max_seq_len
        self.device = device

        self.texts = []
        self.labels = []
        for item in data:
            label = item['label']
            tokens = [token2idx.get(token.lower(), UNK_ID) for token in item['tokens'].split('|')]
            tokens = torch.tensor(tokens, dtype=torch.long, device=self.device)
            self.texts.append(tokens)

            # Transform the continuous label into five classes and add it to the self.labels list
            ### YOUR CODE STARTS HERE
            ...
            ### YOUR CODE ENDS HERE

    def __getitem__(self, idx):
        padded_text = torch.zeros(self.max_seq_len, dtype=torch.long, device=self.device)
        text = self.texts[idx][:self.max_seq_len]
        padded_text[:text.size(0)] = text
        return padded_text, self.labels[idx]

    def __len__(self):
        return len(self.texts)

Load the datasets.

In [ ]:
train_dataset = SSTDataset(sst['train'], token2idx, max_seq_len=52, device=device)
validation_dataset = SSTDataset(sst['validation'], token2idx, max_seq_len=52, device=device)
test_dataset = SSTDataset(sst['test'], token2idx, max_seq_len=52, device=device)

In [ ]:
len(train_dataset), len(validation_dataset), len(test_dataset)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=50)
validation_dataloader = DataLoader(validation_dataset, batch_size=50)
test_dataloader = DataLoader(test_dataset, batch_size=50)

### Task 2. Modify the Model (1 point)

Since now we have five classes instead of two, you need to modify the final layer of the model to have five outputs (ref. [nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html?highlight=linear#torch.nn.Linear)).

Also, we are going to finetune the pretrained embeddings this time, so you need to "unfreeze" them (ref. [nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding)).

This task doesn't have `### YOUR CODE STARTS HERE` field, you have to find the parameters and modify them yourself.

In [ ]:
class SSTClassificationModel(nn.Module):
    def __init__(self, pretrained_emb, num_filters, kernel_sizes):
        super().__init__()
        self.emb = nn.Embedding.from_pretrained(pretrained_emb, padding_idx=PAD_ID, freeze=True)
        emb_size = self.emb.weight.size(1)

        self.convs = nn.ModuleList(
            [
                  nn.Conv1d(in_channels=emb_size, out_channels=num_filters, kernel_size=kernel_size)
                  for kernel_size in kernel_sizes                  
            ]
        )
        self.linear_out = nn.Linear(in_features=num_filters * len(kernel_sizes), out_features=2)
        self.drop = nn.Dropout(0.5)

    def forward(self, x):
        # x size is [batch x seq_len]
        x = self.emb(x) # [batch x seq_len x emb_dim]
        x = x.permute(0, 2, 1) # [batch x emb_dim x seq_len]
        xs = [torch.relu(conv(x)) for conv in self.convs] # [batch x num_filters x conv_seq_len] x num_kernels
        xs = [torch.nn.functional.max_pool1d(x, x.size(2)).squeeze(2) for x in xs] # [batch x num_filters] x num_kernels
        x = torch.cat(xs, dim=1) # [batch x num_filters * num_kernels]
        x = self.drop(x)
        x = self.linear_out(x) # [batch x 2]
        return x

Model parameters. Feel free to modify them if you'd like to.

In [ ]:
num_filters = 100
kernel_sizes = [3, 4, 5]
lr = 1e-3

num_iters = 100

Initalize the model.

In [ ]:
model = SSTClassificationModel(glove_vecs, num_filters, kernel_sizes)
model = model.to(device)

In [ ]:
print(model)

Initialize the loss and optimizer. Feel free to use different optimizer if you'd like to.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Train the model.

__This may take around 25-30 minutes!__

In the end, you should have around 0.40 accuracy.
If the training takes too long or you have time constrains, feel free to reduce the number of epochs.

In [ ]:
best_accuracy = 0.0
for i in range(num_iters):
    current_loss = 0
    model.train()
    for texts, labels in train_dataloader:
        model.zero_grad()
        preds = model(texts)
        loss = loss_fn(preds, labels)
        current_loss += loss
        loss.backward()
        optimizer.step()
    avg_train_loss = current_loss.item() / len(train_dataloader)

    current_loss = 0
    current_acc = 0
    model.eval()
    for texts, labels in validation_dataloader:
        with torch.no_grad():
            preds = model(texts)
            loss = loss_fn(preds, labels)
            preds = torch.argmax(torch.log_softmax(preds, dim=1), dim=1)
            acc = accuracy(preds, labels)
            current_loss += loss
            current_acc += acc
    avg_val_loss = current_loss.item() / len(validation_dataloader)
    avg_val_acc = current_acc.item() / len(validation_dataloader)

    if avg_val_acc > best_accuracy:
        print(f'Accuracy increased [{best_accuracy:.4f} --> {avg_val_acc:.4f}]. Saving the model...')
        best_accuracy = avg_val_acc
        torch.save(model, 'model_best.pt')

    print(f'Epoch: {i}\tTrain loss: {avg_train_loss:.4f}\tVal loss: {avg_val_loss:.4f}\tVal acc: {avg_val_acc:.4f}')

Load the best model.

In [ ]:
model = torch.load('model_best.pt')

In [ ]:
model.to(device)

Test the model on the test set and save all the labels and predictions.

In [ ]:
current_acc = 0
current_loss = 0
y_true = []
y_pred = []
model.eval()
with torch.no_grad():
    for text, label in test_dataloader:
        label = label.to(torch.long)
        preds = model(text)
        loss = loss_fn(preds, label)
        preds = torch.argmax(torch.log_softmax(preds, dim=1), dim=1)
        acc = accuracy(preds, label)
        current_loss += loss
        current_acc += acc
        y_true.append(label)
        y_pred.append(preds)
avg_test_loss = current_loss.item() / len(test_dataloader)
avg_test_acc = current_acc.item() / len(test_dataloader)

print(f'Test loss: {avg_test_loss:.6f}\tTest acc: {avg_test_acc:.2%}')

y_true = torch.hstack(y_true).cpu().numpy()
y_pred = torch.hstack(y_pred).cpu().numpy()

### Task 3. Evaluate the model (1 point)

One way to better understand a multi-class model is to build a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix#Confusion_matrices_with_more_than_two_categories).

Run the cell below and describe what you see. For example, which classes are the most difficult for the model to differentiate? What one are the easiest? Why do you think it happens?

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

class_labels = ['very negative', 'negative', 'neutral', 'positive', 'very positive']
ConfusionMatrixDisplay.from_predictions(y_true, y_pred, display_labels=class_labels, xticks_rotation="vertical");

__YOUR ANSWER HERE__

### Task 4. Fine-tuned embeddings (1 point)

Since we fine-tuned the pretrained word embeddings, they should now better fit our task, which is sentiment classification.

Try different words that have sentimental meaning, e.g. "bad", "good", "fantastic", "disgusting", and find similar words to them using both pretrained GloVe and fine-tuned embeddings. 

Briefly describe which words you tried and which difference you observed. Provide some examples.

In [ ]:
def get_word_embedding(word, embeddings, vocab):
    return embeddings[vocab[word]]

def find_most_similar(word, embeddings, token2idx, idx2token, n=5):
    word_norm = torch.nn.functional.normalize(get_word_embedding(word, embeddings, token2idx), dim=0)
    emb_norm = torch.nn.functional.normalize(embeddings, dim=1)
    similarities = torch.matmul(word_norm, emb_norm.T)
    top_n = torch.argsort(similarities, descending=True)[1:n+1]
    
    print(f'Top-{n} most similar to the word "{word}":\n')
    for idx in top_n:
        print(f'{idx2token[idx]}\t{similarities[idx]:.4f}')

In [ ]:
# Pretrained Glove embeddings
find_most_similar('bad', glove_vecs, token2idx, idx2token)

In [ ]:
# Fine-tuned embeddings
find_most_similar('bad', model.emb.weight, token2idx, idx2token)

__YOUR ANSWER HERE__